# AI Community @ Семинар  №3
## Pandas

**Pandas** - это библиотека Python, предоставляющая широкие возможности для анализа данных. С ее помощью очень удобно загружать, обрабатывать и анализировать табличные данные с помощью SQL-подобных запросов.  
В связке с библиотеками Matplotlib и Seaborn появляется возможность удобного визуального анализа табличных данных. Мы посмотрим на них позже.

In [2]:
# Отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

# Знакомые нам вещи
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Стандартное сокращение для pandas - pd
import pandas as pd

Основными структурами данных в **Pandas** являются классы **Series** и **DataFrame**.  
Первый из них представляет собой одномерный индексированный массив данных некоторого фиксированного типа. Мы можем думать о Series как о векторе из прошлого занятия.  
Второй - это двухмерная структура данных (матрица), представляющая собой таблицу, каждый столбец которой содержит данные одного типа. Можно представлять её как словарь объектов типа Series.  
Структура DataFrame отлично подходит для представления реальных данных: строки соответствуют признаковым описаниям отдельных объектов, а столбцы соответствуют признакам.

Для начала рассмотрим простые примеры создания таких объектов и возможных операций над ними.

### Series

Создание объекта Series из 4 элементов, индексированных словами:

In [4]:
salaries = pd.Series([80000, 65000, 20000, 75000], 
              index = ['Андрей', 'Владимир', 'Чарльз', 'Анна']) 
print(salaries)        

Андрей      80000
Владимир    65000
Чарльз      20000
Анна        75000
dtype: int64


Посмотрим на среднюю зарплату.  
Функции numpy принимают pd.Series, так как для него они выглядят как np.array:

In [6]:
np.mean(salaries)

60000.0

Можно сделать то же самое, обратившись к самому объекту pd.Series:

In [7]:
salaries.mean()

60000.0

Посмотрим на людей, чья зарплата выше средней:

In [ ]:
salaries[salaries > salaries.mean()]

Мы можем обращаться к элементам pd.Series как `salaries['Name']` или `salaries.Name`. Например:

In [11]:
salaries.Андрей, salaries['Андрей']

(80000, 80000)

Можно добавлять новые элементы, обращаясь к несуществующему элементу:

In [12]:
salaries['Кот'] = 300000
salaries

Андрей       80000
Владимир     65000
Чарльз       20000
Анна         75000
Кот         300000
dtype: int64

Индексом может быть строка, состоящая из нескольких слов.  
Также, значением в pd.Series может быть `None`, точнее, его аналог в `numpy - np.nan` (not a number):  

In [14]:
salaries['Сергей Бабочка'] = np.nan
salaries

Андрей             80000.0
Владимир           65000.0
Чарльз             20000.0
Анна               75000.0
Кот               300000.0
Сергей Бабочка         NaN
dtype: float64

В данных часто бывают пропуски, поэтому вы часто будете видеть `np.nan`.  
Важно уметь находить их и обрабатывать.  
Получим битовую маску для пропущенных значений: 

In [15]:
salaries.isnull()

Андрей            False
Владимир          False
Чарльз            False
Анна              False
Кот               False
Сергей Бабочка     True
dtype: bool

In [16]:
salaries[salaries.isnull()]

Сергей Бабочка   NaN
dtype: float64

Назначим минимальную зарплату всем, у кого ее нет (в данном случае только Сергею Бабочке): 

In [17]:
salaries[salaries.isnull()] = 1
salaries

Андрей             80000.0
Владимир           65000.0
Чарльз             20000.0
Анна               75000.0
Кот               300000.0
Сергей Бабочка         1.0
dtype: float64

В дальнейшем мы рассмотрим другие способы обработки пропусков в данных.

### Dataframe

Создадим pd.DataFrame из единичной numpy-матрицы:

In [32]:
df1 = pd.DataFrame(np.eye(3), index=['a', 'b', 'c'], 
                   columns=['col1', 'col2', 'col3'])
df1

,col1,col2,col3
a,1.0,0.0,0.0
b,0.0,1.0,0.0
c,0.0,0.0,1.0


Можно создавать pd.DataFrame из словаря.  
Ключами будут названия столбцов, а значениями - списки значений в этих столбцах.  
pd.DataFrame может хранить значения любых типов. Но в пределах одного столбца тип может быть только один:  

In [33]:
dictionary = {
    'A': np.arange(5),
    'B': ['a', 'b', 'c', 'd', 'e'],
    'C': np.arange(5) > 2
}
df2 = pd.DataFrame(dictionary)
df2

,A,B,C
0,0,a,False
1,1,b,False
2,2,c,False
3,3,d,True
4,4,e,True


Можем обращаться к отдельному элементу в таблице через `at` (это быстро):

In [34]:
df2.at[3, 'B']

'd'

Можем обращаться к куску таблицы через loc (это всего лишь в [22 раза медленнее](https://stackoverflow.com/questions/37216485/pandas-at-versus-loc), чем at):

In [35]:
df2.loc[3:4, ['A', 'B']]

,A,B
3,3,d
4,4,e


Обращение только к строке:

In [36]:
# Нумерация начинается с 0
df2.loc[2]

A        2
B        c
C    False
Name: 2, dtype: object

Обращение только к столбцу.
Если мы хотим выделить все элементы по какой-то координате, можно написать просто `':'`:

In [37]:
# Здесь мы хотим взять все строки и пишем для этого ':'
df2.loc[:, 'B']

0    a
1    b
2    c
3    d
4    e
Name: B, dtype: object

Можем изменять элементы, обращаясь к ним через `at` и присваивая значение:

In [38]:
df2.at[3, 'B'] = 'Z'
df2

,A,B,C
0,0,a,False
1,1,b,False
2,2,c,False
3,3,Z,True
4,4,e,True


С помощью loc можно изменять сразу всю строку.  
И даже создавать новые, смотрите:

In [39]:
df2.loc[5] = [77, '!', False]
df2

,A,B,C
0,0,a,False
1,1,b,False
2,2,c,False
3,3,Z,True
4,4,e,True
5,77,!,False


Создадим копию нашей таблицы без последнего столбца.  
Затем, присоединим новую таблицу к старой и посмотрим, что будет.

In [42]:
df3 = df2.copy().loc[:, ['A', 'B']]
df3 = df2.append(df3)
df3

,A,B,C
0,0,a,False
1,1,b,False
2,2,c,False
3,3,Z,True
4,4,e,True
5,77,!,False
0,0,a,NaN
1,1,b,NaN
2,2,c,NaN
3,3,Z,NaN


Jupyter автоматически выводит последнее значение в ячейке.  
Многие методы pandas не изменяют оригинальную таблицу, а возвращают копию.  
Давайте выкинем все строки, в которых есть NaN:

In [45]:
df3.dropna()

,A,B,C
0,0,a,False
1,1,b,False
2,2,c,False
3,3,Z,True
4,4,e,True
5,77,!,False


Вместо строк можно убрать столбцы, в которых есть NaN.  
Для этого нужно передать параметр `axis=1`:

In [47]:
df3.dropna(axis=1)

,A,B
0,0,a
1,1,b
2,2,c
3,3,Z
4,4,e
5,77,!
0,0,a
1,1,b
2,2,c
3,3,Z


В результате выполнения этих методов наша таблица не изменилась, потому что возвращалась копия.  
Заменим все NaN каким-то значением:

In [50]:
df3.fillna(False)

,A,B,C
0,0,a,False
1,1,b,False
2,2,c,False
3,3,Z,True
4,4,e,True
5,77,!,False
0,0,a,False
1,1,b,False
2,2,c,False
3,3,Z,False


### Пример первичного анализа данных с Python

На практике данные приходится считывать из файла. `pandas` умеет считывать `csv` файлы (comma-separated values) с помощью метода **read_csv()**. Такие файлы состоят из набора строк, в каждой из которых находятся значения признаков, разделенные запятой (или другим разделителем).    
  
Рассмотрим работу с `pd.DataFrame` на примере следующего набора данных. Для каждого опрошенного имеется следующая информация: заработная плата за час работы, опыт работы, образование, субъективная внешняя привлекательность (в баллах от 1 до 5), бинарные признаки: пол, семейное положение, состояние здоровья (хорошее/плохое), членство в профсоюзе, цвет кожи (белый/чёрный), занятость в сфере обслуживания (да/нет).

In [51]:
# В данном случае данные разделены знаком ';' и мы явно это указываем
df = pd.read_csv('data/beauty.csv', sep = ';')

Посмотрим на размерность данных (shape).  
Первое значение - количество строк (примеров), второе - количество признаков:

In [52]:
df.shape

(1260, 10)

Посмотрим на первые 3 элемента таблицы с помощью метода __head()__:

In [53]:
# По умолчанию аргументом является число 5. 
# То есть, если вызвать df.head(), то покажется 5 строк.
df.head(3)

,wage,exper,union,goodhlth,black,female,married,service,educ,looks
0,5.73,30,0,1,0,1,1,1,14,4
1,4.28,28,0,1,0,1,1,0,12,3
2,7.96,35,0,1,0,1,0,0,10,4


Метод **describe()** показывает основные статистические характеристики данных по каждому признаку: число непропущенных значений, среднее, стандартное отклонение, диапазон, медиану, 0.25 и 0.75 квартили.

In [54]:
df.describe()

,wage,exper,union,goodhlth,black,female,married,service,educ,looks
count,1260.000000,1260.000000,1260.000000,1260.000000,1260.000000,1260.000000,1260.000000,1260.000000,1260.000000,1260.000000
mean,6.306690,18.206349,0.272222,0.933333,0.073810,0.346032,0.691270,0.273810,12.563492,3.185714
std,4.660639,11.963485,0.445280,0.249543,0.261564,0.475892,0.462153,0.446089,2.624489,0.684877
min,1.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,1.000000
25%,3.707500,8.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,12.000000,3.000000
50%,5.300000,15.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,12.000000,3.000000
75%,7.695000,27.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,13.000000,4.000000
max,77.720000,48.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,17.000000,5.000000


Отсортируем значения в таблице по размеру заработной платы:

In [57]:
df.sort_values(by='wage', ascending=False).head(3)

,wage,exper,union,goodhlth,black,female,married,service,educ,looks
602,77.72,9,1,1,1,1,1,1,13,4
269,41.67,16,0,0,0,0,1,0,13,4
415,38.86,29,0,1,0,0,1,0,13,3


Можно сортировать по нескольким признакам сразу:

In [59]:
df.sort_values(by=['goodhlth', 'wage'], ascending=[False, True]).head(3)

,wage,exper,union,goodhlth,black,female,married,service,educ,looks
1214,1.02,11,0,1,0,1,1,1,13,3
1009,1.05,29,0,1,1,0,1,0,5,3
1226,1.09,8,0,1,0,1,1,1,10,2


Индексировать pd.DataFrame можно по-разному.  
Чтобы извлечь один столбец (признак), можно обратиться к нему как к элементу словаря df['Name'].  
Извлечем признак, отвечающий за здоровье и посмотрим на среднее значение этого признака. Это скажет нам, какова доля людей с хорошим здоровьем среди опрошенных:

In [60]:
df['goodhlth'].mean()

0.93333333333333335

Чтобы получить битовую маску для какого-то столбца, можно сравнить столбец с каким-то значением:

In [62]:
(df['female'] == 1).head()

0     True
1     True
2     True
3    False
4    False
Name: female, dtype: bool

Затем, эту битовую маску можно передать в таблицу, чтоб получить те строки, где значение в маске равно `True`:

In [63]:
df[df['female'] == 1].head()

,wage,exper,union,goodhlth,black,female,married,service,educ,looks
0,5.73,30,0,1,0,1,1,1,14,4
1,4.28,28,0,1,0,1,1,0,12,3
2,7.96,35,0,1,0,1,0,0,10,4
5,3.91,20,0,0,0,1,1,0,12,3
8,5.00,5,0,1,0,1,0,0,16,3


Посмотрим, сколько в нашей таблице мужчин с плохим здоровьем:

In [64]:
df[(df['female'] == 0) & (df['goodhlth'] == 0)].shape

(49, 10)

Очень часто необходимо посмотреть, сколько различных значений есть в столбце, и сколько строк (примеров) соответствует каждому из них:

In [65]:
df['married'].value_counts()

1    871
0    389
Name: married, dtype: int64

Можно применить какую-нибудь функцию к каждому столбцу (признаку).  
Например, найдем максимальное значение для каждого признака:

In [67]:
df.apply(np.max)

wage        77.72
exper       48.00
union        1.00
goodhlth     1.00
black        1.00
female       1.00
married      1.00
service      1.00
educ        17.00
looks        5.00
dtype: float64